In [4]:

%cd /home/plent/Documenten/Gitlab/NeuralODEs/jax_neural_odes
import amici
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

model_name = "Smallbone2013_SerineBiosynthesis"
model_names_pypesto_dir = "smallbone"

output_dir = f"results/timing_benchmark/pypesto_compiled_models/{model_names_pypesto_dir}/"
model_module = amici.import_model_module(model_name, output_dir)
model = model_module.getModel()

# model.setParameterByName(parameter_name, value)

/home/plent/Documenten/Gitlab/NeuralODEs/jax_neural_odes


In [5]:
model_names = ["Smallbone2013_SerineBiosynthesis","Becker_Science2010", "Fujita_SciSignal2010","Raia_CancerResearch","Messiha2013"]
model_names_pypesto_dir=["smallbone","becker","fujita","raia","messiha"]

parameter_initializations_dirs = [
    "parameter_initializations/Smallbone2013_SerineBiosynthesis/Smallbone2013_SerineBiosynthesis_parameterset_id_lhs_N=100run_1bounds_10.csv",
    "parameter_initializations/Becker_Science2010/Becker_Science2010_parameterset_id_lhs_N=100run_1bounds_10.csv",
    "parameter_initializations/Fujita_SciSignal2010/Fujita_SciSignal2010_parameterset_id_lhs_N=100run_1bounds_10.csv",
    "parameter_initializations/Raia_CancerResearch/Raia_CancerResearch_parameterset_id_lhs_N=100run_1bounds_10.csv",
    "parameter_initializations/Messiha2013/Messiha2013_parameterset_id_lhs_N=100run_1bounds_10.csv"

]

dataset_dirs = [
    "datasets/Smallbone2013 - Serine biosynthesis/Smallbone2013 - Serine biosynthesis_dataset.csv",
    "datasets/Becker_Science2010/Becker_Science2010_dataset.csv",
    "datasets/Fujita_SciSignal2010/Fujita_SciSignal2010_dataset.csv",
    "datasets/Raia_CancerResearch/Raia_CancerResearch_dataset.csv",
    "datasets/Messiha2013/Messiha2013_dataset.csv",
]



In [283]:
n_reps = 10
sensitivity_methods=["forward","adjoint"]


timing_dict={}

for sensitivity_method in sensitivity_methods:
    for k,model_name in enumerate(model_names):
        parameter_inits = pd.read_csv(parameter_initializations_dirs[k],index_col=0)
        dataset = pd.read_csv(dataset_dirs[k],index_col=0)

        # parameter_inits = pd.read_csv("parameter_initializations/Smallbone2013_SerineBiosynthesis/"
        #                               "Smallbone2013_SerineBiosynthesis_parameterset_id_lhs_N=100run_1bounds_10.csv",index_col=0)
        # dataset = pd.read_csv("datasets/Smallbone2013 - Serine biosynthesis/"
        #                       "Smallbone2013 - Serine biosynthesis_dataset.csv",index_col=0)

        output_dir = f"results/timing_benchmark/pypesto_compiled_models/{model_names_pypesto_dir[k]}/"
        model_module = amici.import_model_module(model_name, output_dir)
        model = model_module.getModel()

        ts=dataset.index.to_list()
        model.setTimepoints(ts)
        solver = model.getSolver()
        # solver.setLinearSolver(amici.LinearSolver_dense)
        solver.setAbsoluteTolerance(1e-12)
        solver.setRelativeTolerance(1e-12)
        solver.setMaxSteps(maxsteps=300000)

        model.requireSensitivitiesForAllParameters()

        if sensitivity_method == "adjoint":
            solver.setSensitivityMethod(amici.SensitivityMethod_adjoint)
        elif sensitivity_method == "forward":
            solver.setSensitivityMethod(amici.SensitivityMethod_forward)

        solver.setSensitivityOrder(1)


        rdata = amici.runAmiciSimulation(model, solver)


        timings=[]
        for i in range(n_reps):
            for k in range(np.shape(parameter_inits)[0]):
                parameter_set  = parameter_inits.iloc[k,:]
                for i in parameter_set.keys():
                    value = parameter_set.get(i)
                    if "lp." in i:
                        i = i.replace("lp.","")
                        i = i.replace(".","_")
                        try:
                            model.setParameterByName(i, value)
                        except:
                            print(model_name,i)

                start= time.perf_counter()
                edata= amici.ExpData(np.shape(rdata.y)[1],
                                     0,
                                     0,
                                     rdata.ts)
                for i in range(np.shape(rdata.y)[1]):
                    edata.setObservedData(rdata.y[:,i],i)

                rdata = amici.runAmiciSimulation(model, solver,edata)
                grads = rdata.sllh

                end= time.perf_counter()
                timings.append(end-start)
        print(f"{model_name}", np.mean(timings))
        timing_dict[f"{model_name}_amici_{sensitivity_method}"] = {"mean": np.mean(timings), "std": np.std(timings)}




results = {}
for key, value in timing_dict.items():
    model, solver = key.rsplit("_", 1)
    if model not in results:
        results[model] = {}
    results[model][solver] = f"{value['mean']:.4f} ± {value['std']:.4f}"

df = pd.DataFrame(results).T  # Transpose to get models as rows
df.index.name = "Model"
df.columns.name = "Solver"

df.to_csv("results/timing_benchmark/time_benchmark_backward_amici.csv")




Smallbone2013_SerineBiosynthesis 0.01490284414589405
Becker_Science2010 0.007563441168022109
Fujita_SciSignal2010 0.03375930980202975
Raia_CancerResearch 0.02223484725636081
Messiha2013 1.3979815172256493
Smallbone2013_SerineBiosynthesis 0.010936258902569534
Becker_Science2010 0.0014295104161719792
Fujita_SciSignal2010 0.0028364455153932796
Raia_CancerResearch 0.0033820676848117727
Messiha2013 0.41885685367518455


In [284]:

results = {}
for key, value in timing_dict.items():
    model, solver = key.rsplit("_", 1)
    if model not in results:
        results[model] = {}
    results[model][solver] = f"{value['mean']:.4f} ± {value['std']:.4f}"

df = pd.DataFrame(results).T  # Transpose to get models as rows
df.index.name = "Model"
df.columns.name = "Solver"

df.to_csv("results/timing_benchmark/time_benchmark_backward_amici.csv")

# Jaxkineticmodel grads

In [1]:
import diffrax

In [6]:
from jaxkineticmodel.load_sbml.sbml_model import SBMLModel
import jax.numpy as jnp
import pandas as pd
import jax

jax.config.update("jax_enable_x64", True)



In [7]:
def mse_loss_func(params, ts, ys, kmodel):
    """A typical mean squared error loss function"""
    y0 = ys[0, :]
    y_pred = kmodel(ts, y0, params)
    loss = jnp.sum((y_pred - ys) ** 2)
    return loss

def wrapped_loss(params, ts, ys):
    return mse_loss_func(params, ts, ys, JaxKmodel)


model_names = ["Smallbone2013_SerineBiosynthesis","Becker_Science2010", "Fujita_SciSignal2010","Raia_CancerResearch","Messiha2013"]

n_reps=1
for k,model_name in enumerate(model_names):


    filepath = (f"models/sbml_models/working_models/{model_name}.xml")
    parameter_inits = pd.read_csv(parameter_initializations_dirs[k],index_col=0)
    dataset = pd.read_csv(dataset_dirs[k],index_col=0)



    ts=dataset.index.to_list()
    # load model from file_path
    model = SBMLModel(filepath)

    #replaces assignment rules, boundary conditions, etc..

    S=model._get_stoichiometric_matrix()
    JaxKmodel = model.get_kinetic_model()
    JaxKmodel._change_solver(solver=diffrax.Dopri5(), rtol=1e-12, atol=1e-12,dtol=1e-8,icoeff=0.1)

    try:

        loss, grads = jax.value_and_grad(wrapped_loss, 0)(model.parameters, ts, jnp.array(np.array(dataset)))



        timings=[]
        for i in range(n_reps):
            print(i)
            for k in range(np.shape(parameter_inits)[0]):
                parameter_set  = parameter_inits.iloc[k,:].to_dict()
                start= time.perf_counter()
                loss, grads = jax.value_and_grad(wrapped_loss, 0)(model.parameters, ts, jnp.array(np.array(dataset)))
                end= time.perf_counter()
                timings.append(end-start)
        print(f"{model_name}", np.mean(timings))
        timing_dict[f"{model_name}_jaxkineticmodel_dopri"] = {"mean": np.mean(timings), "std": np.std(timings)}
    except:
        continue


17:15:06,275 - jax._src.xla_bridge - INFO - Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
17:15:06,278 - jax._src.xla_bridge - INFO - Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
0
Smallbone2013_SerineBiosynthesis 0.01998769628989976
0
Becker_Science2010 0.016748073361814022
0
Raia_CancerResearch 0.04349284098832868
0


In [357]:
timing_dict

{'Smallbone2013_SerineBiosynthesis_amici_forward': {'mean': 0.01490284414589405,
  'std': 0.005877856013384586},
 'Becker_Science2010_amici_forward': {'mean': 0.007563441168022109,
  'std': 0.0034650659306721303},
 'Fujita_SciSignal2010_amici_forward': {'mean': 0.03375930980202975,
  'std': 0.012625524242626687},
 'Raia_CancerResearch_amici_forward': {'mean': 0.02223484725636081,
  'std': 0.0020737654513462624},
 'Messiha2013_amici_forward': {'mean': 1.3979815172256493,
  'std': 0.39099632742488966},
 'Smallbone2013_SerineBiosynthesis_amici_adjoint': {'mean': 0.010936258902569534,
  'std': 0.004055327172083837},
 'Becker_Science2010_amici_adjoint': {'mean': 0.0014295104161719792,
  'std': 0.0002762280963900279},
 'Fujita_SciSignal2010_amici_adjoint': {'mean': 0.0028364455153932796,
  'std': 0.0005591707438727477},
 'Raia_CancerResearch_amici_adjoint': {'mean': 0.0033820676848117727,
  'std': 0.00038557628651518825},
 'Messiha2013_amici_adjoint': {'mean': 0.41885685367518455,
  'std': 0

In [359]:
results = {}
for key, value in timing_dict.items():
    model, solver = key.rsplit("_", 1)
    if model not in results:
        results[model] = {}
    results[model][solver] = f"{value['mean']:.4f} ± {value['std']:.4f}"

df = pd.DataFrame(results).T  # Transpose to get models as rows
df.index.name = "Model"
df.columns.name = "Solver"

df['dopri']

Model
Smallbone2013_SerineBiosynthesis_amici                           NaN
Becker_Science2010_amici                                         NaN
Fujita_SciSignal2010_amici                                       NaN
Raia_CancerResearch_amici                                        NaN
Messiha2013_amici                                                NaN
Smallbone2013_SerineBiosynthesis                                 NaN
Becker_Science2010                                               NaN
Fujita_SciSignal2010                                             NaN
Raia_CancerResearch                                              NaN
Messiha2013                                                      NaN
Smallbone2013_SerineBiosynthesis_jaxkineticmodel     0.0240 ± 0.0015
Becker_Science2010_jaxkineticmodel                   0.0183 ± 0.0026
Raia_CancerResearch_jaxkineticmodel                  0.0368 ± 0.0049
Messiha2013_jaxkineticmodel                         14.4700 ± 1.5348
Name: dopri, dtype: object